# Copy攻城狮信手”粘“来 AI 对对联

> 源码来源：https://www.kesci.com/mw/project/5c47088b2d8ef5002b737590

> 数据集： https://github.com/wb14123/couplet-dataset.git

本案例基于 ModelArts 我的笔记本模块实现，使用 GPU 环境的 TensorFlow 1.13.1 ，使用 [Seq2Seq](https://github.com/google/seq2seq) 实现 对对联模型。

## 依赖安装及引用

```
!pip install klab-autotime
!pip install backcall
```

In [1]:
!pip install klab-autotime
!pip install backcall

You are using pip version 9.0.1, however version 21.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 21.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import codecs
import numpy as np
from keras.models import Model
from keras.layers import *
from keras.callbacks import Callback

Using TensorFlow backend.
/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [3]:
# 显示cell运行时长
%load_ext klab-autotime

# 使用GPU
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

## 数据处理
couplet-dataset 尽管比较陈旧，但拥有 70 万条数据，应该够实现一个简单的对对联模型。

In [14]:
# 下载对联数据集
!wget https://github.com/wb14123/couplet-dataset/releases/download/1.0/couplet.tar.gz
!tar -xzvf couplet.tar.gz.1
!mkdir couplet/model

couplet/
couplet/train/
couplet/train/in.txt
couplet/train/out.txt
couplet/test/
couplet/test/in.txt
couplet/test/out.txt
couplet/test/.out.txt.swp
couplet/test/.in.txt.swp
couplet/vocabs
time: 890 ms


In [4]:
min_count = 2
maxlen = 16
batch_size = 64
char_size = 128


time: 1.21 ms


In [5]:
train_input_path = 'couplet/train/in.txt'
train_output_path = 'couplet/train/out.txt'
test_input_path = 'couplet/test/in.txt'
test_output_path = 'couplet/test/out.txt'

time: 1.21 ms


In [6]:
# 数据读取与切分
def read_data(txtname):
    txt = codecs.open(txtname, encoding='utf-8').readlines()
    txt = [line.strip().split(' ') for line in txt]      # 每行按空格切分
    txt = [line for line in txt if len(line) <= maxlen]  # 过滤掉字数超过maxlen的对联
    return txt

time: 2.85 ms


In [7]:
# 训练数据的前10行上联
txt_sample = codecs.open(train_input_path, encoding='utf-8').readlines()
txt_sample[:10]

['晚 风 摇 树 树 还 挺 \n',
 '愿 景 天 成 无 墨 迹 \n',
 '丹 枫 江 冷 人 初 去 \n',
 '忽 忽 几 晨 昏 ， 离 别 间 之 ， 疾 病 间 之 ， 不 及 终 年 同 静 好 \n',
 '闲 来 野 钓 人 稀 处 \n',
 '毋 人 负 我 ， 毋 我 负 人 ， 柳 下 虽 和 有 介 称 ， 先 生 字 此 ， 可 以 谥 此 \n',
 '投 石 向 天 跟 命 斗 \n',
 '深 院 落 滕 花 ， 石 不 点 头 龙 不 语 \n',
 '不 畏 鸿 门 传 汉 祚 \n',
 '新 居 落 成 创 业 始 \n']

time: 150 ms


In [8]:
# 经过切分后的训练数据的前10行上联
x_train_txt = read_data(train_input_path)
x_train_txt[:10]  # 查看前10行

[['晚', '风', '摇', '树', '树', '还', '挺'],
 ['愿', '景', '天', '成', '无', '墨', '迹'],
 ['丹', '枫', '江', '冷', '人', '初', '去'],
 ['闲', '来', '野', '钓', '人', '稀', '处'],
 ['投', '石', '向', '天', '跟', '命', '斗'],
 ['深', '院', '落', '滕', '花', '，', '石', '不', '点', '头', '龙', '不', '语'],
 ['不', '畏', '鸿', '门', '传', '汉', '祚'],
 ['新', '居', '落', '成', '创', '业', '始'],
 ['本', '领', '高', '强', '攀', '月', '桂'],
 ['豪', '华', '超', '御', '苑']]

time: 2.18 s


In [9]:
# 经过切分后的训练数据的前10行下联
y_train_txt = read_data(train_output_path)
y_train_txt[:10]

[['晨', '露', '润', '花', '花', '更', '红'],
 ['万', '方', '乐', '奏', '有', '于', '阗'],
 ['绿', '柳', '堤', '新', '燕', '复', '来'],
 ['兴', '起', '高', '歌', '酒', '醉', '中'],
 ['闭', '门', '问', '卷', '与', '时', '争'],
 ['残', '经', '凋', '贝', '叶', '，', '香', '无', '飞', '篆', '磬', '无', '声'],
 ['难', '堪', '垓', '下', '别', '虞', '姬'],
 ['宏', '图', '初', '振', '治', '家', '先'],
 ['成', '亲', '吉', '利', '放', '兰', '香'],
 ['康', '乐', '驻', '山', '城']]

time: 2.34 s


In [10]:
# 同样的，剩余的测试集数据也如上处理
x_test_txt = read_data(test_input_path)
y_test_txt = read_data(test_output_path)

time: 44.5 ms


In [11]:
# 将对联按字数分组
# 记录每个字词在所有对联中的出现次数
chars = {}

for txt in [x_train_txt,y_train_txt,x_test_txt,y_test_txt]:
    for line in txt:
        for word in line:
            chars[word] = chars.get(word,0) + 1 #如果字典中不包含key,默认值为0

time: 3.13 s


In [12]:
# 查看字的计数字典数据样例
c = 0
for word,count in chars.items():
    if c <=5:
        print(word,count)
    c = c+1

晚 5960
风 120861
摇 5579
树 15825
还 7033
挺 974
time: 4.29 ms


生成 id:字，字:id 对的字典
- chars:过滤掉只出现了1次的字后的 字:次数 字典
- id2char：(id:字) 对的字典
- char2id:(字:id) 对的字典

In [13]:
# 过滤掉只出现了1次的字
chars = {word:count for word,count in chars.items() if count >= min_count}

# word_id:word
# {1: '晚',
# 2: '风',...}
id2char = {word_id+1:word for word_id,word in enumerate(chars)}

# 更换一下key-value的位置
# word:word_id
#{'晚': 1,
#'风': 2,
#'摇': 3,...}
char2id = {word:word_id for word_id,word in id2char.items()}

time: 15.2 ms


In [14]:
# 将字匹配对应id
def string2id(char_list):
    # 0: <unk>
    return [char2id.get(char,0) for char in char_list]

time: 1.45 ms


In [15]:
# 输出前5行上联的每个字匹配成了以下的id
x_train = list(map(string2id, x_train_txt))
# x_train = list(map(lambda char_list:[char2id.get(char,0) for char in char_list], x_train_txt))
x_train[:5]

[[1, 2, 3, 4, 4, 5, 6],
 [7, 8, 9, 10, 11, 12, 13],
 [14, 15, 16, 17, 18, 19, 20],
 [21, 22, 23, 24, 18, 25, 26],
 [27, 28, 29, 9, 30, 31, 32]]

time: 1.76 s


In [16]:
# 输出前5行下联的每个字匹配成了以下的id
y_train = list(map(string2id, y_train_txt))
y_train[:5]

[[1568, 281, 666, 37, 37, 280, 435],
 [98, 254, 936, 1525, 141, 586, 4379],
 [489, 96, 756, 50, 486, 1736, 22],
 [354, 310, 57, 491, 188, 187, 136],
 [1421, 46, 183, 634, 449, 91, 450]]

time: 2.34 s


In [17]:
# 对剩余的测试集也做以上匹配处理
x_test = list(map(string2id, x_test_txt))
y_test = list(map(string2id, y_test_txt))

time: 17.1 ms


In [18]:
# 按字数分组存放上联与下联数据
# 按字数分组存放于字典中,每个字数是一个字典,存放[样本数据,类别标记]列表

def generate_count_dict(result_dict,x,y):
    for i,charIDlist in enumerate(x):
        j = len(charIDlist)
        if j not in result_dict:
            result_dict[j] = [[],[]]  # [样本数据list,类别标记list]
        result_dict[j][0].append(charIDlist)
        result_dict[j][1].append(y[i])
    return result_dict

time: 5.02 ms


In [19]:
# train_dict = {字数1,字数2,..}
train_dict = {}
test_dict = {}

train_dict = generate_count_dict(train_dict, x_train, y_train)
test_dict = generate_count_dict(test_dict,x_test,y_test)

time: 213 ms


In [20]:
# 查看train_dict中的内容信息
print('共有{}中不同的字数'.format(len(train_dict.keys())))
for wordCount,[data,y] in train_dict.items():
    print('字数',wordCount,':对应上联x的个数:',len(data),'下联y的个数',len(y))

共有16中不同的字数
字数 7 :对应上联x的个数: 356042 下联y的个数 356042
字数 13 :对应上联x的个数: 27982 下联y的个数 27982
字数 5 :对应上联x的个数: 75951 下联y的个数 75951
字数 15 :对应上联x的个数: 11723 下联y的个数 11723
字数 12 :对应上联x的个数: 91220 下联y的个数 91220
字数 14 :对应上联x的个数: 9304 下联y的个数 9304
字数 3 :对应上联x的个数: 5949 下联y的个数 5949
字数 11 :对应上联x的个数: 16944 下联y的个数 16944
字数 4 :对应上联x的个数: 13831 下联y的个数 13831
字数 10 :对应上联x的个数: 17770 下联y的个数 17770
字数 6 :对应上联x的个数: 10223 下联y的个数 10223
字数 9 :对应上联x的个数: 24970 下联y的个数 24970
字数 16 :对应上联x的个数: 7437 下联y的个数 7437
字数 2 :对应上联x的个数: 12185 下联y的个数 12185
字数 1 :对应上联x的个数: 2313 下联y的个数 2313
字数 8 :对应上联x的个数: 1726 下联y的个数 1726
time: 58.3 ms


In [21]:
#将训练集与测试集中的x与y np.array数组化
def toNumpyArray(d):
    for count,[data,y] in d.items():
        d[count][0] = np.array(data)
        d[count][1] = np.array(y)
    return d

time: 2.38 ms


In [22]:
train_dict = toNumpyArray(train_dict)
test_dict = toNumpyArray(test_dict)

time: 1.78 s


In [23]:
# 查看字数为7的训练集上联与下联的数组
train_dict[7]

[array([[   1,    2,    3, ...,    4,    5,    6],
        [   7,    8,    9, ...,   11,   12,   13],
        [  14,   15,   16, ...,   18,   19,   20],
        ...,
        [1562,  931,  405, ...,  996, 2138,  291],
        [1005, 1430,  425, ..., 2186, 1737,  663],
        [ 578,  764,  135, ...,  501,  984,   60]]),
 array([[1568,  281,  666, ...,   37,  280,  435],
        [  98,  254,  936, ...,  141,  586, 4379],
        [ 489,   96,  756, ...,  486, 1736,   22],
        ...,
        [ 173,  798,  663, ..., 1146,   84, 2182],
        [ 820, 1150,  158, ...,  402,  328,  178],
        [1289, 1588, 1638, ..., 1578,   10,  417]])]

time: 4.82 ms


In [24]:
# 随机抽取生成大小为batch的上联与下联数据集
# data: train_dict 或者 test_dict

def data_generator(data):
    # 计算每个对联长度的权重
    data_probability = [float(len(x)) for wordcount,[x,y] in data.items()] # [每个字数key对应对联list中上联数据的个数]
    data_probability = np.array(data_probability) / sum(data_probability)  # 标准化至[0,1]，这是每个字数的权重
    
    # 随机选择字数，然后随机选择字数对应的上联样本，生成batch
    while True: 
        # 随机选字数id，概率为上面计算的字数权重
        idx = np.random.choice(len(data_probability), p=data_probability) + 1
        size = min(batch_size, len(data[idx][0])) # batch_size=64，len(data[idx][0])随机选择的字数key对应的上联个数
        
        # 从上联列表下标list中随机选出大小为size的list
        idxs = np.random.choice(len(data[idx][0]), size = size)
        
        # 返回选出的上联X与下联y, 将原本1-d array维度扩展为(row,col,1)
        yield data[idx][0][idxs], np.expand_dims(data[idx][1][idxs],axis=2)
        # return data[idx][0][idxs], np.expand_dims(data[idx][1][idxs],axis=2)
    

time: 13.4 ms


In [25]:
data_generator(test_dict)

<generator object data_generator at 0x7fb7eedbbeb8>

time: 2.06 ms


## 模型构建

In [26]:
def gated_resnet(x, ksize=3):
    # 门卷积 + 残差
    x_dim = K.int_shape(x)[-1]
    xo = Conv1D(x_dim*2, ksize, padding='same')(x)
    return Lambda(lambda x: x[0] * K.sigmoid(x[1][..., :x_dim]) \
                            + x[1][..., x_dim:] * K.sigmoid(-x[1][..., :x_dim]))([x, xo])

time: 2.74 ms


In [27]:
# 模型代码
x_in = Input(shape=(None,))
x = x_in
x = Embedding(len(chars)+1, char_size)(x)
x = Dropout(0.25)(x)

x = gated_resnet(x)
x = gated_resnet(x)
x = gated_resnet(x)
x = gated_resnet(x)
x = gated_resnet(x)
x = gated_resnet(x)

x = Dense(len(chars)+1, activation='softmax')(x)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
time: 272 ms


In [28]:
model = Model(x_in, x)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam')

time: 33.7 ms


In [29]:
# 下联输出函数
def couplet_match(s):
    # 输出对联
    # 先验知识：跟上联同一位置的字不能一样
    x = np.array([string2id(s)]) # 上联-->id array
    y = model.predict(x)[0]
    
    for i,j in enumerate(x[0]):
        y[i, j] = 0.
        
    y = y[:, 1:].argmax(axis=1) + 1
    r = ''.join([id2char[i] for i in y])
    
    print('上联：%s，下联：%s' % (s, r))
    return r

time: 7.52 ms


In [30]:
# 评估函数
class Evaluate(Callback):
    def __init__(self):
        self.lowest = 1e10
    def on_epoch_end(self, epoch, logs=None):
        
        # 训练过程中观察几个例子，显示对联质量提高的过程
        couplet_match(u'晚风摇树树还挺')
        couplet_match(u'今天天气不错')
        couplet_match(u'鱼跃此时海')
        couplet_match(u'只有香如故')
        
        # 保存最优结果
        if logs['val_loss'] <= self.lowest:
            self.lowest = logs['val_loss']
            model.save_weights('couplet/model/best_model.weights') # 保存模型的权重
            model.save('couplet/model/my_couplet_model') # 保存模型

time: 7.76 ms


In [31]:
# 训练
evaluator = Evaluate()

model.fit_generator(data_generator(train_dict),
                    steps_per_epoch=1000,
                    epochs=200,
                    validation_data=data_generator(test_dict),
                    validation_steps=200,
                    callbacks=[evaluator])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/200
1000/1000 [==============================] - 12s 12ms/step - loss: 6.9415 - val_loss: 7.4614
上联：晚风摇树树还挺，下联：风月落人风月行
上联：今天天气不错，下联：月月月风无人
上联：鱼跃此时海，下联：月生月月心
上联：只有香如故，下联：春风月月人
Epoch 2/200
1000/1000 [==============================] - 6s 6ms/step - loss: 5.7305 - val_loss: 8.1736
上联：晚风摇树树还挺，下联：中月映春风不成
上联：今天天气不错，下联：春地无心无来
上联：鱼跃此时海，下联：马、大世风
上联：只有香如故，下联：无无月有春
Epoch 3/200
1000/1000 [==============================] - 6s 6ms/step - loss: 4.5987 - val_loss: 8.4387
上联：晚风摇树树还挺，下联：夕月润春人不生
上联：今天天气不错，下联：大水月人无来
上联：鱼跃此时海，下联：鸟开大处人
上联：只有香如故，下联：无如水有春
Epoch 4/200
1000/1000 [==============================] - 6s 6ms/step - loss: 3.9344 - val_loss: 8.1723
上联：晚风摇树树还挺，下联：红雨落风月不飞
上联：今天天气不错，下联：大日古人无来
上联：鱼跃此时海，下联：马开大世天
上联：只有香如故，下联：无无水有人
Epoch 5/200
1000/1000 [==============================] - 6s 6ms/step - loss: 3.4696 - val_loss: 7.8751
上联：晚风摇树树还挺，下联：红雨舞花月不飞
上联：今天天气不错，下联：大日人人无心
上联：

time: 20min 48s


In [33]:
# 加载训练好的模型

from keras.models import load_model
new_model = load_model('couplet/model/my_couplet_model')

time: 1.88 s


In [34]:
def predict_couplet(model,s):
    x = np.array([string2id(s)]) # 上联-->id array
    y = new_model.predict(x)[0]
    
    for i,j in enumerate(x[0]):
        y[i, j] = 0.
        
    y = y[:, 1:].argmax(axis=1) + 1
    r = ''.join([id2char[i] for i in y])
    
    print('上联：%s \n下联：%s' % (s, r))

time: 7.07 ms


In [35]:
# 输出对联
s = u'天增岁月人增寿'
predict_couplet(new_model,s)
s = u'鼠去牛来闻虎啸'
predict_couplet(new_model,s)
s = u'流光溢彩气冲斗牛'
predict_couplet(new_model,s)

上联：天增岁月人增寿 
下联：国满春秋我成春
上联：鼠去牛来闻虎啸 
下联：羊来马去看龙吟
上联：流光溢彩气冲斗牛 
下联：春色流辉风震春虫
time: 184 ms
